In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)

In [3]:
file_paths = {'in':'../../../data/prepared/activity_log_train.csv',
              'out':''
             }

df = pd.read_csv(file_paths['in'])

# guarantee the timestamps are datetime objects
time_columns = ['start_time','end_time']

for time_col in time_columns:
    df[time_col] = pd.to_datetime(df[time_col])

In [4]:
df.head()

,start_time,ride_id,row_count,segment_count,end_time,elapsed_time,moving_time,elapsed_distance,elapsed_ascent,elapsed_descent,max_speed,avg_speed,ride_cruise_speed,avg_power,ride_avg_power,ride_max_power,peak_20min_power,training_window_id,perceived_exertion,simple_exertion,perceived_relative_effort,ride_work,hours_since_last_ride,active_time_ratio,ftp_power,intensity,time_intensity
0,2020-09-02 21:21:17+00:00,4003175803,4224,15,2020-09-02 22:46:42+00:00,5151,4173,14.666919,611.2503,612.5627,28.632832,12.522739,14.541428,95.573578,112.724170,1058.440085,146.082425,4,5.0,NORMAL,75.0,580.642200,75.300833,0.810134,146.082425,0.771648,46.488211
1,2020-10-24 20:44:22+00:00,4238817655,2097,5,2020-10-24 21:22:49+00:00,2308,2058,7.747065,192.2666,146.9888,22.145705,13.312794,15.123428,103.136627,118.232006,606.592362,144.190970,5,7.0,HARD,65.0,272.879469,0.018333,0.891681,144.460600,0.818438,0.014446
2,2021-09-20 11:41:28+00:00,5990296028,2013,4,2021-09-20 12:17:01+00:00,2134,1985,7.445936,201.7815,148.6293,25.053728,13.363364,15.723231,113.923630,129.781023,1067.198334,136.550865,11,8.0,HARD,85.0,276.952703,59.486389,0.930178,169.020795,0.767841,35.729477
3,2020-10-16 19:04:26+00:00,4203579387,3169,8,2020-10-16 20:06:37+00:00,3786,3114,11.225301,318.9132,320.8818,20.579848,12.767843,14.524287,98.027222,104.373331,522.192986,128.441399,5,7.0,HARD,99.0,395.157430,459.581944,0.822504,144.460600,0.722504,292.237704
4,2020-08-06 22:28:05+00:00,3875911617,1159,11,2020-08-06 23:46:19+00:00,4720,1141,3.171579,78.4159,58.0737,29.751302,9.914663,15.002448,110.379005,94.070193,615.164105,NaN,3,5.0,NORMAL,20.0,444.011312,4.995833,0.241737,161.323024,0.583117,3.202162


In [5]:
df.shape

(126, 27)

# 1. Feature Selection